## Assignment 5
Topics: Decision Trees, Random Forests, and Boosting ##



In [13]:
# some basic imports
from scipy import io
import pandas as pd
import numpy as np
import math
import re
import matplotlib.pyplot as plt
import copy
from functools import partial
from multiprocessing.dummy import Pool as ThreadPool

%matplotlib inline
%load_ext autoreload
%autoreload 2

#loading training dataset
train = pd.read_csv(r'C:\Users\syeda\Desktop\Machine Learning\Assignment-5\train.csv')
#loading test dataset
test = pd.read_csv(r'C:\Users\syeda\Desktop\Machine Learning\Assignment-5\test.csv')

#displaying the shapes of the train and test data
print("train shape: {} test shape: {}".format(train.shape, test.shape))
#top 3 elements of train
train.head(3)

#top 3 elements of test
test.head(3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
train shape: (1047, 11) test shape: (262, 11)


,Pclass,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,0,"Hart, Mr. Henry",male,NaN,0,0,394140,6.8583,NaN,Q
1,3,0,"Johnston, Master. William Arthur ""Willie""",male,NaN,1,2,W./C. 6607,23.4500,NaN,S
2,3,0,"Goodwin, Master. William Frederick",male,11.0,5,2,CA 2144,46.9000,NaN,S


Referencing from https://www.kaggle.com/competitions/titanic/overview

In [14]:

original_train = train.copy() # Using 'copy()' allows to clone the dataset, creating a different object with the same values

# Feature engineering steps taken from Sina and Anisotropic, with minor changes to avoid warnings
full_data = [train, test]

# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

# Remove all NULLS in the Age column
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    # Next line has been improved to avoid warning
    dataset.loc[np.isnan(dataset['Age']), 'Age'] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)

# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Master": 2, "Mrs": 3, "Miss": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] ;

In [15]:
drop_elements = ['Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
test  = test.drop(drop_elements, axis = 1)

## Visualization of  the  processed data from titanic dataset ##

In [16]:
train.head(3)

,Pclass,Survived,Sex,Age,Parch,Fare,Embarked,Has_Cabin,FamilySize,IsAlone,Title
0,1,1,0,1,0,3,1,1,1,1,4
1,3,0,1,2,0,0,0,0,1,1,1
2,3,0,1,2,0,1,0,0,3,0,1


In [17]:
val = "Survived"
#var variable is selected to be dropped from the table
X = train.drop([val], axis=1)
X_test = test.drop([val], axis=1)
y = train[val]
y_test = test[val]
print("train: {}, test: {}".format(X.shape, X_test.shape))
print("train: {}, test: {}".format(y.shape, y_test.shape))
print((np.array(X)).shape[1])
print((np.array(y)).shape[0])

train: (1047, 10), test: (262, 10)
train: (1047,), test: (262,)
10
1047


In [18]:
def accuracy(val_1, val_2):
    #accuracy is calculated as a sum when two values are equal and its divided my the length of the shape
    acc = np.sum(val_1 == val_2) / val_1.shape[0]
    return acc
print("Individaul Survived: {:.6f}, Individual Not Survivied: {:.6f}".format(y.sum()/len(y), 1-y.sum()/len(y)))

    

Individaul Survived: 0.387775, Individual Not Survivied: 0.612225


# Implementing Random Forest 
Also adding input parameters:
criterion - With Gini impurity, or entropy.,
max_depth, 
min_samples_split, 
min_samples_leaf.

In [19]:
# class for decision tree
class DecisionTree:
     #defining initialization for decision tree
    def __init__(self, criterion,m_dpth, min_smpls_leaf,sampl_featr=False):
         #criterion for selecting  information_gain, gini_impurity
        if criterion == 'entropy':
            self.criterion = self._information_gain
        elif criterion == 'gini':
            self.criterion = self._gini_purification
        else:
            print('Criterion is information_gain_ratio or gini or entropy')
        self._tree, self.sampl_featr, self.m_dpth, self.min_samples_leaf = None, sampl_featr, m_dpth, min_smpls_leaf
        print(type(min_smpls_leaf))

  # defining the fit method for the decision tree
    def fit(self, X, y, sampl_wt=None):
        if sampl_wt is None:
            sampl_wt = np.ones(X.shape[0]) / X.shape[0]
        else:
            sampl_wt = np.array(sampl_wt) / np.sum(sampl_wt)

        featr_names, X, y = X.columns.tolist(), np.array(X), np.array(y)
        self._tree = self._build_tree(X, y, featr_names, depth=1, sample_weights=sampl_wt)
        return self
        
   # defining entropy as static method 
    @staticmethod
    def entropy(y, sampl_wt=None):
        entrop, num = 0.0, y.shape[0]
        labelCts={}
        for i in range(num):
            if y[i] not in labelCts.keys():
                labelCts[y[i]]=0
            labelCts[y[i]]+=sampl_wt[i]
        for key in labelCts:
            entrop-=float(labelCts[key])/float(np.sum(sampl_wt))*np.log2(float(labelCts[key])/float(np.sum(sampl_wt)))
        return entrop

  #defining information gain 
    def _information_gain(self, X, y, indx, sampl_wt=None):
        in_gain, old_ct, uniq_val, new_ct = 0, self.entropy(y, sampl_wt), np.unique(X[:,indx]), 0.0
        for value in uniq_val:
            sub_X,sub_y, sub_sampl_wt=self._split_dataset(X, y, indx, value, sampl_wt)
            prob=np.sum(sub_sampl_wt)/float(np.sum(sampl_wt))
            new_ct+=prob*self.entropy(sub_y, sub_sampl_wt)
        in_gain=old_ct-new_ct 
        return in_gain

   # defining gini_impurity as static method with labelcounts
    @staticmethod
    def gini_impurity(y, sampl_wt=None):
        gini_i, num = 1, y.shape[0]
        labelCts={}
        for i in range(num):
            if y[i] not in labelCts.keys():
                labelCts[y[i]]=0
            labelCts[y[i]]+=sampl_wt[i]
        for key in labelCts:
            gini_i -= (float(labelCts[key])/float(np.sum(sampl_wt))) ** 2
        return gini_i

  #defining gini_purification
    def _gini_purification(self, X, y, indx, sampl_wt=None):
        n_impurity, old_ct, uniq_val, new_ct = 0, self.gini_impurity(y, sampl_wt), np.unique(X[:,indx]), 0.0
        for value in uniq_val:
            sub_X,sub_y, sub_sampl_wt=self._split_dataset(X, y, indx, value, sampl_wt)
            new_ct+=np.sum(sub_sampl_wt)/float(np.sum(sampl_wt))*self.gini_impurity(sub_y, sub_sampl_wt)
        n_impurity=old_ct-new_ct 
        return n_impurity

 #defining split dataset method
    def _split_dataset(self, X, y, indx, val, sampl_wt=None):
        rt, ftVec=[],X[:,indx]
        X=X[:,[i for i in range(X.shape[1]) if i!=indx ]]
        for i in range(len(ftVec)):
            if ftVec[i]==val:
                rt.append(i)
        sub_X, sub_y, sub_sampl_wt = X[rt,:], y[rt], sampl_wt[rt]
        return sub_X, sub_y, sub_sampl_wt

 #defining to choose best features
    def _choose_best_feature(self, X, y, sampl_wt=None):
        best_featr_idx, n_featr = 0, X.shape[1]
        if self.sampl_featr:
            max_featr=max(1, min(n_featr, int(np.round(np.sqrt(n_featr)))))
            new_featr=np.random.choice(n_featr, max_featr, replace=False)
            new_X=X[:, new_featr]
        else:
            new_X=X
        n_new_featr, best_gain_ct=new_X.shape[1], 0.0
        for i in range(n_new_featr):
            info_gain_ct=self.criterion(new_X,y,i,sampl_wt)           
            if info_gain_ct > best_gain_ct:
                best_gain_ct, best_featr_idx=info_gain_ct, i          
        return best_featr_idx
        
 #defining majority vote as static method
    @staticmethod
    def majority_vote(y, sampl_wt=None):
        if sampl_wt is None:
            sampl_wt = np.ones(y.shape[0]) / y.shape[0]
        majority_lbl = y[0]
        dict_num={}
        for i in range(y.shape[0]):
            if int(y[i]) not in dict_num.keys():
                dict_num[y[i]]=sampl_wt[i]
            else:
                dict_num[y[i]] += sampl_wt[i]
        return max(dict_num, key=dict_num.get)

 #defining build tree for construction of decision tree
    def _build_tree(self, X, y, feature_names, depth, sample_weights=None):
        mytree = dict()
        if len(feature_names)==0 or len(np.unique(y))==1 or int(depth) >= int(self.m_dpth) or int(len(X)) <= int(self.min_samples_leaf): 
            return self.majority_vote(y, sample_weights)
        best_feature_idx=self._choose_best_feature(X, y, sample_weights)
        best_feature_name = feature_names[best_feature_idx]
        feature_names=feature_names[:]
        feature_names.remove(best_feature_name)
        mytree={best_feature_name:{}}
        unique_vals=np.unique(X[:, best_feature_idx])
        for value in unique_vals:
            sub_X, sub_y, sub_sample_weights = self._split_dataset(X, y, best_feature_idx, value, sample_weights)
            mytree[best_feature_name][value]=self._build_tree(sub_X, sub_y, feature_names, depth+1, sub_sample_weights)
        return mytree
 #defining predict method for decision trees
    def predict(self, X):
         #defining classify method
        def _classify(tree, x):
            featr_name=list(tree.keys())[0]
            s_dict, key=tree[featr_name], x.loc[featr_name]         
            if key not in s_dict:
                key=np.random.choice(list(s_dict.keys()))
            valueOfKey=s_dict[key]
            return isinstance(valueOfKey,dict) and _classify(valueOfKey,x) or valueOfKey
        if len(X.shape)==1:
            return _classify(self._tree,X)
        else:
            results=[]
            for i in range(X.shape[0]):
                results.append(_classify(self._tree, X.iloc[i, :]))
            return np.array(results)




## Decision Tree Model Implementation  ##

In [20]:
 # implementing model of decision tree by calling decision tree class
dt = DecisionTree(criterion='gini', m_dpth=4, min_smpls_leaf=4, sampl_featr=False)
dt.fit(X, y)
y_train_pred = dt.predict(X)
 # printing accuracy on train of the model generated
print("Accuracy on train set: {}".format(accuracy(y, y_train_pred)))
# printing accuracy on test for the model generated
print("Accuracy on test set: {}".format(accuracy(y_test, dt.predict(X_test))))



<class 'int'>
Accuracy on train set: 0.2817574021012416
Accuracy on test set: 0.25190839694656486


# Implementing Random Forest 

In [21]:
 #defining class random forest
class RandomForest:
    def __init__(self, tree_learner,n_estimator,state=2020):
        np.random.seed(state)
        self.n_estimator, self.weak_learner= n_estimator, tree_learner
        self._estimators = [copy.deepcopy(self.weak_learner) for _ in range(self.n_estimator)]

  #defining function to get bootstrap dataset
    def _get_bootstrap_dataset(self, X, y, i):
        X_arr, y_arr=np.array(X), np.array(y)
        idx=np.random.choice(X_arr.shape[0], X_arr.shape[0], replace=True)
        X_bootstrap= X.iloc[idx, :]
        y_bootstrap = y.iloc[idx]
        self._estimators[i].fit(X_bootstrap, y_bootstrap)

  #defining fit for random forest classifier
    def fit(self, X, y):
        self.labels, part_btstrap, pool=list(set(y)), partial(self._get_bootstrap_dataset, X, y), ThreadPool(self.n_estimator)
        pool.map(part_btstrap, list(range(self.n_estimator)))
        pool.close() and pool.join()
        return self

  # defining predict for random forest classifier 
    def predict(self, X):
        N = X.shape[0]
        y_pred = np.zeros(N)
        preds=np.array([estimator.predict(X) for estimator in self._estimators])
        pred_proba=np.array([(preds == label).mean(axis=0) for label in self.labels]).T
        y_pred=np.array(self.labels)[np.argmax(pred_proba, axis=1)]
        return y_pred


## Random forest Model Implementation  ##

In [22]:
# implementing model for random forest
base_learner = DecisionTree(criterion='gini', m_dpth=6, min_smpls_leaf=6, sampl_featr=True)
r = RandomForest(tree_learner=base_learner, n_estimator=100, state=2020)
r.fit(X, y)
#predicting the training set
y_train_pred = r.predict(X)
# printing accuracy on train for the model generated
print("Accuracy on train set: {}".format(accuracy(y, y_train_pred)))
# printing accuracy on test for the model generated
print("Accuracy on test set: {}".format(accuracy(y_test, r.predict(X_test))))



<class 'int'>
Accuracy on train set: 0.6571155682903533
Accuracy on test set: 0.6412213740458015


# Implementing AdaBoost as a second classifier(BOOSTING)

In [23]:
 # defining the boosting class
class Adaboost:
    # defining the initialization of method for boosting
    def __init__(self,weak_learner,n_estimator,state=2020):
        np.random.seed(state)
        self.weak_learner, self.n_estimator = weak_learner, n_estimator
        self._estimators = [copy.deepcopy(self.weak_learner) 
                            for _ in range(self.n_estimator)]
        self._alphas = [1 
                        for _ in range(n_estimator)]
    # defining the fit method for boosting 
    def fit(self, X, y):
        L=y.shape[0]
        w=np.ones(L)/L
        for i in range(self.n_estimator):
            self._estimators[i].fit(X, y, sampl_wt=w)
            pred=self._estimators[i].predict(X)
            err=np.sum((pred!=y)*w)
            alpha=0.5*np.log((1-err)/err)
            self._alphas[i]=alpha
            w*=np.exp(-alpha*np.array(y)*pred) + 1e-4
            w/=2*np.sqrt(err*(1-err))
        return self
  #defining the predict method for boosting
    def predict(self, X):
        N = X.shape[0]
        y_pred = np.zeros(N)
        for i in range(self.n_estimator):
            y_pred += self._alphas[i]*self._estimators[i].predict(X)   
        return np.sign(y_pred)


## Adaboost Model Implementation  ##

In [24]:
 # implementing the model for adabboost boosting classifier
base_learner = DecisionTree(criterion='entropy', m_dpth=2, min_smpls_leaf=1, sampl_featr=False)
ada = Adaboost(weak_learner= base_learner, n_estimator=50, state=2020)
ada.fit(X, y)
#predicting the training set
y_train_pred = ada.predict(X)
# printing accuracy on train for the model generated
print("Accuracy on train set: {}".format(accuracy(y, y_train_pred)))
# printing accuracy on test for the model generated
print("Accuracy on test set: {}".format(accuracy(y_test, ada.predict(X_test))))


<class 'int'>
Accuracy on train set: 0.7793696275071633
Accuracy on test set: 0.767175572519084


# Table of results

In [25]:
print('Model performance on Decision tree\n')
 # printing accuracy on train of the model generated
print("Accuracy on train set: {}".format(accuracy(y, y_train_pred)))
# printing accuracy on test for the model generated
print("Accuracy on test set: {}".format(accuracy(y_test, dt.predict(X_test))))
print('\nModel performance on Random Forest\n')
# printing accuracy on train for the model generated
print("Accuracy on train set: {}".format(accuracy(y, y_train_pred)))
# printing accuracy on test for the model generated
print("Accuracy on test set: {}".format(accuracy(y_test, r.predict(X_test))))
print('\nModel performance on Boosting(ADAboost)\n')
# printing accuracy on train for the model generated
print("Accuracy on train set: {}".format(accuracy(y, y_train_pred)))
# printing accuracy on test for the model generated
print("Accuracy on test set: {}".format(accuracy(y_test, ada.predict(X_test))))


Model performance on Decision tree

Accuracy on train set: 0.7793696275071633
Accuracy on test set: 0.25190839694656486

Model performance on Random Forest

Accuracy on train set: 0.7793696275071633
Accuracy on test set: 0.6412213740458015

Model performance on Boosting(ADAboost)

Accuracy on train set: 0.7793696275071633
Accuracy on test set: 0.767175572519084
